# Импорты

In [ ]:
import subprocess
import sys

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    subprocess.run(["pip", "install", "catboost>=1.2.7"])

# Для более качественных графиков
%config InlineBackend.figure_format='retina'
plt.rcParams["figure.dpi"] = 150

# Датасет California Housing Prices

## Загрузка данных

[Ссылка на файл датасета](https://drive.google.com/file/d/1SCYD2qDlrybLVfiFObKJS9bwUPJLpcIW/view?usp=sharing)

[Информация о датасете](https://www.kaggle.com/datasets/camnugent/california-housing-prices)

In [ ]:
df = pd.read_csv("data/housing.csv")
target = "median_house_value"
df

## EDA

In [ ]:
df.info()

In [ ]:
df_info = df.describe()
df_info

### Пропущенные значения

In [ ]:
df["total_bedrooms"].isna().sum()

Имеем 207 пропущенных значений в столбце total_bedrooms

In [ ]:
bad_col = "total_bedrooms"

df_nan_info = df.loc[df[bad_col].isna()].describe()

nan_std = df_nan_info.loc["std"]
std = df_info.loc["std"]

df_nan_info

In [ ]:
print("Относительная разница между подвыборкой с нанами и со всей выборкой")
((df_nan_info - df_info) / df_nan_info).loc[["mean", "std"]]

In [ ]:
mean_rooms = df_info.loc["mean", "total_rooms"]
mean_rooms_nan = df_nan_info.loc["mean", "total_rooms"]
print(f"Кол-во комнат в среднем: {mean_rooms:48.3f}")
print(f"Кол-во комнат в среднем для пропущенных значений total_bedrooms: {mean_rooms_nan:.3f}")

Объекты с пропущенными значениями не особо отличаются от остальных. Способов работы с пропущенными значениями много, мы возьмем самый простой - заменим их на 0

### Визуализация данных

In [ ]:
X = df.drop(columns=target)

num_cols = list(X.select_dtypes(exclude=object).columns)
cat_cols = list(X.select_dtypes(include=object).columns)
g = sns.PairGrid(df, x_vars=num_cols + [target], y_vars=num_cols + [target])
g.map_lower(sns.scatterplot)
plt.tight_layout()
plt.show()

In [ ]:
for col in cat_cols:
    plt.figure(figsize=(3, 2))
    sns.boxplot(df, x=target, y=col)
    plt.title(f"Распределение таргета при разных значениях категорий {col}")
    plt.show()

В зависимости от категории распределение таргета сильно меняется. Использование этого признака довольно перспективно (чтобы узнать, как использовать категориальные признаки в моделях, можно найти соответствующий материал в интернете)

In [ ]:
ax = df.hist(bins=60, figsize=(7, 7))
plt.suptitle("Распределение числовых признаков")
plt.show()

### Аномалии

Можно заметить неожиданные пики для признаков `median_house_value` и `housing_median_age`. Давайте рассмотрим их поподробнее.

`median_house_value`:

In [ ]:
print("Самые частые значения:")
df["median_house_value"].value_counts()

In [ ]:
weird_value = 500001.0
df_weird_info = df[df["median_house_value"] == weird_value].describe()
df_weird_info

In [ ]:
((df_weird_info - df_info) / df_weird_info).loc[["mean", "std"]]

Вероятно это максимально допустимое значение признака, и если значение больше, то оно приравнивается к 500001.0

При этом факт того, что этих значений так много, довольно подозрителен

`housing_median_age`:

In [ ]:
df["housing_median_age"].value_counts()

In [ ]:
weird_value = 52.0

df_weird_info = df[df["housing_median_age"] == weird_value].describe()
df_weird_info

In [ ]:
((df_weird_info - df_info) / df_weird_info).loc[["mean", "std"]]

Предположение относительно `housing_medium_age` аналогично прошлому признаку, при этом аномальное значение тут тоже самое частое

### Корреляции

In [ ]:
df_corr = df[num_cols + [target]].corr()
mask = np.triu(df_corr).astype(bool)
sns.heatmap(df_corr, annot=True, mask=mask)
plt.show()

## Обучение

Выделим таргет и признаки для обучения модели

In [ ]:
X = df.drop(columns=[target, "ocean_proximity"])
X = X.fillna(0)
y = df[target]

Обучим линейную регрессию

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X, y)
pred = model.predict(X)
pred

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error


def model_summary(y_true, y_pred):
    mae = mean_absolute_error(y_true=y_true, y_pred=y_pred)
    mape = mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred)
    print(f"Средняя абсолютная ошибка: {mae:.3f}")
    print(f"Средняя абсолютная процентная ошибка: {mape:.3f}")

    plt.scatter(x=y_pred, y=y_true, label="Реальное предсказание")
    plt.plot(y_true, y_true, linestyle="--", color="orange", label="Идеальное предсказание")
    plt.xlabel("Предсказание")
    plt.xlabel("Таргет")
    plt.legend()
    plt.show()


model_summary(y_true=y, y_pred=pred)

Сделаем тестовую выборку

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)

In [ ]:
model_summary(y_true=y_test, y_pred=pred)

Быстрый способ построить качественную модель - использовать бустинг (а точнее кэтбуст - он поддерживает наны и категориальные значения в признаках):

In [ ]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(cat_features=cat_cols, allow_writing_files=False)

X = df.drop(columns=target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train, verbose=False)
pred = model.predict(X_test)
model_summary(y_true=y_test, y_pred=pred)

# Базовые метрики для регрессии:

- Корень средняя квадратичной ошибки (RMSE): потенциальная проблема - масштаб ошибки зависит от данных
- Средняя абсолютная ошибка (MAE): аналогично RMSE
- Средняя абсолютная процентная ошибка (MAPE): удобно, так как мы видим, как в среднем прогноз отклоняется в процентах от правильного ответа
- R2: какая доля вариации объясняется предсказанием. Обычно используется в эконометрике, на мой взгляд в реальных ситуациях не нужна

RMSE, MAE и MAPE удобны своей простотой и позволяют быстро оценить качество модели, однако для потребностей бизнеса они часто недостаточны: в реальности нужно делать метрику, которая показывают пользу модели при использовании на практике. Например, если мы делаем модель для решения, предоставлять ли клиенту банка кредит, то метрикой будет средняя прибыль от кредитов при использовании модели. 

Часто такую метрику может быть сложно подсчитать, но нужно попытаться сделать такую метрику, которая будет как можно ближе к запросу бизнеса 

# Метрики для бинарной классификации:

- Доля правильных ответов (accuracy - часто называют как "точность")
- Точность (precision): расчитывается по формуле $\frac{TP}{TP+FP}$, где FP (False Positive) - доля неправильных положительных ответов, TP (True Positive) - доля правильных положительных ответов
- Полнота (recall): расчитывается по формуле $\frac{TP}{TP+FN}$, где FN (False Negative) - доля неправильных отрицательных ответов
- F1: $\frac{2 * precision * recall}{precision + recall}$ - среднее гармоническое точности и полноты

В классификации полезно исследовать матрицу ошибок (confusion matrix):

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


preds = np.array([0, 1, 1, 1])
target = np.array([0, 1, 0, 1])
ConfusionMatrixDisplay.from_predictions(y_true=target, y_pred=preds)
plt.show()

# Задания для самостоятельного решения

Ссылки:
- [датасет](https://drive.google.com/file/d/1SDBuPOtEYY5vqtbCrkpsgoGjucNVYFWr/view?usp=sharing)
- [информация о датасете](https://www.kaggle.com/datasets/ahsan81/hotel-reservations-classification-dataset)
1. Скачайте датасет, исследуйте его признаки аналогично тому, как это было сделано выше.  
2. Разделите датасет на тренировочную и тестовую выборки с параметрами `random_seed`=420 и `test_size`=0.2, обучите модель классификации, используя модель `CatBoostClassifier` из библиотеки `catboost`. Задача классификации: определить, отменил ли клиент бронь (за это отвечает признак `booking_status`)
3. Посчитайте долю правильных ответов модели на тестовой выборке и выведите матрицу ошибок. Предложите возможное применение этой модели для бизнеса (в чем может быть польза). Как можно построить бизнесовую метрику?